In [8]:
import pandas as pd
import numpy as np
import re
import os
import glob
from sklearn.metrics import mean_squared_error

In [ ]:
# Evaluation metric
# rmse = mean_squared_error(y_actual, y_predicted, squared=False)

# Pre-processing

In [2]:
# Get files to read in
gcse_files = glob.glob("../fake_data/synthetic_*_gcse_20[1-2][0, 8-9].csv")
npd_files = glob.glob("../fake_data/synthetic_npd_ks4_student_20[1-2][0, 8-9].csv")

## Exam Data

In [3]:
def process_grades(data = pd.DataFrame, grade_col = str):
    
    # Drop rows with missing grades
    data = data.dropna(subset = grade_col)
    # Convert U grade to 0
    data.loc[data[grade_col] == "U", grade_col] = "0"
    # Convert grades to numeric from string format
    data = data[data[grade_col].isin([str(x) for x in (range(0, 10))])]
    data[grade_col] = data[grade_col].astype(int)
    return data

In [4]:
def process_gcse_data(df = pd.DataFrame):
    
    """
    Takes raw GCSE exam data (2017-2020 files), filters it
    appropriately and processes it. 
    Returns a DataFrame with a reduced number of columns.
    Full steps taken can be seen in code commenting or in
    Methodology section of capstone.
    --------------------------------------------------
    df = DataFrame of raw GCSE data
    """
    
    # Copy to prevent in-place changes
    data = df.copy()
    
    # Make cols lowercase
    data.columns = [x.lower() for x in data.columns]
    
    # Reformat examseries to year col
    data["year"] = data.examseries.apply(lambda x: x.split()[1])
    
    # Remove candidates who were not 16 on 31st August
    data = data.query("yearendage == 16")
    # Remove private candidates
    data = data.query("privatecandidate == False")
    # Commented out below since all True in synthetic data
    # Remove partial absentees
#     data = data.query("partialabsence == False")
    # Remove candidates without prior attainment or that weren't matched in NPD
    data = data.dropna(subset = ["normalisedks2score", "npdmatchround"])
    
    # Remove candidates with 0 prior attainment (errors in data)
    data = data[data.normalisedks2score > 0]
    
    # Remove non-reformed GCSEs
    data = data[data.reformphase.isin(['Ofqual-regulated Phase 1 reformed GCSE FC',
                                       'Ofqual-regulated Phase 2 reformed GCSE FC'])]
    # Recode tier into foundation or not foundation
    data.loc[data.tier != "F", "tier"] = "Not F"
    
    # Process grade column inplace
    data = process_grades(data, grade_col = "grade")
    
    # Get candidates who took at least 8 GCSEs
    grouped = data.groupby("uidp").count()
    at_least_8 = set(grouped[grouped.examseries >= 8].index.to_list())
    # Get candidates who took English and Maths
    eng_math = set(data[data.jcqtitle.isin(["Mathematics", "English language"])].uidp)
    # Get candidates who took English and Maths and >= 8 GCSEs
    filtered_ids = at_least_8 & eng_math
    # Beware that since this is simulated data, it's wrong
    filtered = data[data.uidp.isin(filtered_ids)]
    
    # Select cols needed for modelling and dropnas
    gcse_cols = ["uidp", "year", "jcqtitle", "tier", "centretypedesc",
                 "normalisedks2score", "grade", "centreassessmentgrade"]
    filtered = filtered[gcse_cols]

    return filtered

In [5]:
# Load and process all the GCSE exam data
gcse_data = pd.DataFrame()
# Iterate through files
for file in gcse_files:
    # Perform filtering/pre-processing
    year_df = process_gcse_data(pd.read_csv(file))
    # Process the CAG column too
    if "2020" in file:
        year_df = process_grades(year_df, "centreassessmentgrade")
    # Create dummy value for other years
    else:
        year_df.centreassessmentgrade = 999
        
    # Merge with other years
    gcse_data = pd.concat([gcse_data, year_df])
    # Delete var to save memory
    del year_df
# Reset index
gcse_data = gcse_data.reset_index(drop = True)

## NPD Data

In [6]:
gcse_data.shape

(408, 8)

In [7]:
gcse_data.groupby("year").mean()

,normalisedks2score,grade,centreassessmentgrade
year,,,
2018,52.320000,4.906250,999.000000
2019,52.207121,4.878981,999.000000
2020,51.287986,5.429224,5.077626


In [66]:
df = pd.read_csv(npd_files[0])

In [69]:
df.columns

Index(['UIDP', 'KS4_URNP', 'KS2_URNP', 'KS4_NCNP', 'KS4_ACADYR',
       'KS4_AGE_START', 'KS4_YEARGRP', 'KS4_ACTYRGRP', 'KS4_ACTYRGRP_GSIZE',
       'KS4_ACTYRGRP_GSIZE_1',
       ...
       'EnrolStatus_SPR20', 'NCyearActual_SPR20', 'SENprovision_SPR20',
       'SENprovisionMajor_SPR20', 'PrimarySENtype_SPR20',
       'IDACIScore_15_SPR20', 'IDACIRank_15_SPR20', 'IDACIScore_19_SPR20',
       'IDACIRank_19_SPR20', 'UnitContactTime_SPR20'],
      dtype='object', length=521)

In [68]:
[x for x in df.columns if not "KS" in x]

['UIDP',
 'RecordStatus_SPR20',
 'AcademicYear_SPR20',
 'CensusDate_SPR20',
 'CensusTerm_SPR20',
 'OnRoll_SPR20',
 'Phase_SPR20',
 'Gender_SPR20',
 'AgeAtStartOfAcademicYear_SPR20',
 'EthnicGroupMinor_SPR20',
 'EthnicGroupMajor_SPR20',
 'FSMeligible_SPR20',
 'EVERFSM_ALL_SPR20',
 'YSSA_SPR20',
 'LanguageGroupMinor_SPR20',
 'LanguageGroupMajor_SPR20',
 'EnrolStatus_SPR20',
 'NCyearActual_SPR20',
 'SENprovision_SPR20',
 'SENprovisionMajor_SPR20',
 'PrimarySENtype_SPR20',
 'IDACIScore_15_SPR20',
 'IDACIRank_15_SPR20',
 'IDACIScore_19_SPR20',
 'IDACIRank_19_SPR20',
 'UnitContactTime_SPR20']